## Prepare Dataset

In [1]:
! pip install -q accelerate peft lightning nltk

In [2]:
! pip install -q -U bitsandbytes

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
# download dataset

import transformers
import datasets
from pathlib import Path
import os

DATA_REPO_ID = "tanganke/stanford_cars"

class_names = [
    'AM General Hummer SUV 2000',
    'Acura RL Sedan 2012',
    'Acura TL Sedan 2012',
    'Acura TL Type-S 2008',
    'Acura TSX Sedan 2012',
    'Acura Integra Type R 2001',
    'Acura ZDX Hatchback 2012',
    'Aston Martin V8 Vantage Convertible 2012',
    'Aston Martin V8 Vantage Coupe 2012',
    'Aston Martin Virage Convertible 2012',
    'Aston Martin Virage Coupe 2012',
    'Audi RS 4 Convertible 2008',
    'Audi A5 Coupe 2012',
    'Audi TTS Coupe 2012',
    'Audi R8 Coupe 2012',
    'Audi V8 Sedan 1994',
    'Audi 100 Sedan 1994',
    'Audi 100 Wagon 1994',
    'Audi TT Hatchback 2011',
    'Audi S6 Sedan 2011',
    'Audi S5 Convertible 2012',
    'Audi S5 Coupe 2012',
    'Audi S4 Sedan 2012',
    'Audi S4 Sedan 2007',
    'Audi TT RS Coupe 2012',
    'BMW ActiveHybrid 5 Sedan 2012',
    'BMW 1 Series Convertible 2012',
    'BMW 1 Series Coupe 2012',
    'BMW 3 Series Sedan 2012',
    'BMW 3 Series Wagon 2012',
    'BMW 6 Series Convertible 2007',
    'BMW X5 SUV 2007',
    'BMW X6 SUV 2012',
    'BMW M3 Coupe 2012',
    'BMW M5 Sedan 2010',
    'BMW M6 Convertible 2010',
    'BMW X3 SUV 2012',
    'BMW Z4 Convertible 2012',
    'Bentley Continental Supersports Conv. Convertible 2012',
    'Bentley Arnage Sedan 2009',
    'Bentley Mulsanne Sedan 2011',
    'Bentley Continental GT Coupe 2012',
    'Bentley Continental GT Coupe 2007',
    'Bentley Continental Flying Spur Sedan 2007',
    'Bugatti Veyron 16.4 Convertible 2009',
    'Bugatti Veyron 16.4 Coupe 2009',
    'Buick Regal GS 2012',
    'Buick Rainier SUV 2007',
    'Buick Verano Sedan 2012',
    'Buick Enclave SUV 2012',
    'Cadillac CTS-V Sedan 2012',
    'Cadillac SRX SUV 2012',
    'Cadillac Escalade EXT Crew Cab 2007',
    'Chevrolet Silverado 1500 Hybrid Crew Cab 2012',
    'Chevrolet Corvette Convertible 2012',
    'Chevrolet Corvette ZR1 2012',
    'Chevrolet Corvette Ron Fellows Edition Z06 2007',
    'Chevrolet Traverse SUV 2012',
    'Chevrolet Camaro Convertible 2012',
    'Chevrolet HHR SS 2010',
    'Chevrolet Impala Sedan 2007',
    'Chevrolet Tahoe Hybrid SUV 2012',
    'Chevrolet Sonic Sedan 2012',
    'Chevrolet Express Cargo Van 2007',
    'Chevrolet Avalanche Crew Cab 2012',
    'Chevrolet Cobalt SS 2010',
    'Chevrolet Malibu Hybrid Sedan 2010',
    'Chevrolet TrailBlazer SS 2009',
    'Chevrolet Silverado 2500HD Regular Cab 2012',
    'Chevrolet Silverado 1500 Classic Extended Cab 2007',
    'Chevrolet Express Van 2007',
    'Chevrolet Monte Carlo Coupe 2007',
    'Chevrolet Malibu Sedan 2007',
    'Chevrolet Silverado 1500 Extended Cab 2012',
    'Chevrolet Silverado 1500 Regular Cab 2012',
    'Chrysler Aspen SUV 2009',
    'Chrysler Sebring Convertible 2010',
    'Chrysler Town and Country Minivan 2012',
    'Chrysler 300 SRT-8 2010',
    'Chrysler Crossfire Convertible 2008',
    'Chrysler PT Cruiser Convertible 2008',
    'Daewoo Nubira Wagon 2002',
    'Dodge Caliber Wagon 2012',
    'Dodge Caliber Wagon 2007',
    'Dodge Caravan Minivan 1997',
    'Dodge Ram Pickup 3500 Crew Cab 2010',
    'Dodge Ram Pickup 3500 Quad Cab 2009',
    'Dodge Sprinter Cargo Van 2009',
    'Dodge Journey SUV 2012',
    'Dodge Dakota Crew Cab 2010',
    'Dodge Dakota Club Cab 2007',
    'Dodge Magnum Wagon 2008',
    'Dodge Challenger SRT8 2011',
    'Dodge Durango SUV 2012',
    'Dodge Durango SUV 2007',
    'Dodge Charger Sedan 2012',
    'Dodge Charger SRT-8 2009',
    'Eagle Talon Hatchback 1998',
    'FIAT 500 Abarth 2012',
    'FIAT 500 Convertible 2012',
    'Ferrari FF Coupe 2012',
    'Ferrari California Convertible 2012',
    'Ferrari 458 Italia Convertible 2012',
    'Ferrari 458 Italia Coupe 2012',
    'Fisker Karma Sedan 2012',
    'Ford F-450 Super Duty Crew Cab 2012',
    'Ford Mustang Convertible 2007',
    'Ford Freestar Minivan 2007',
    'Ford Expedition EL SUV 2009',
    'Ford Edge SUV 2012',
    'Ford Ranger SuperCab 2011',
    'Ford GT Coupe 2006',
    'Ford F-150 Regular Cab 2012',
    'Ford F-150 Regular Cab 2007',
    'Ford Focus Sedan 2007',
    'Ford E-Series Wagon Van 2012',
    'Ford Fiesta Sedan 2012',
    'GMC Terrain SUV 2012',
    'GMC Savana Van 2012',
    'GMC Yukon Hybrid SUV 2012',
    'GMC Acadia SUV 2012',
    'GMC Canyon Extended Cab 2012',
    'Geo Metro Convertible 1993',
    'HUMMER H3T Crew Cab 2010',
    'HUMMER H2 SUT Crew Cab 2009',
    'Honda Odyssey Minivan 2012',
    'Honda Odyssey Minivan 2007',
    'Honda Accord Coupe 2012',
    'Honda Accord Sedan 2012',
    'Hyundai Veloster Hatchback 2012',
    'Hyundai Santa Fe SUV 2012',
    'Hyundai Tucson SUV 2012',
    'Hyundai Veracruz SUV 2012',
    'Hyundai Sonata Hybrid Sedan 2012',
    'Hyundai Elantra Sedan 2007',
    'Hyundai Accent Sedan 2012',
    'Hyundai Genesis Sedan 2012',
    'Hyundai Sonata Sedan 2012',
    'Hyundai Elantra Touring Hatchback 2012',
    'Hyundai Azera Sedan 2012',
    'Infiniti G Coupe IPL 2012',
    'Infiniti QX56 SUV 2011',
    'Isuzu Ascender SUV 2008',
    'Jaguar XK XKR 2012',
    'Jeep Patriot SUV 2012',
    'Jeep Wrangler SUV 2012',
    'Jeep Liberty SUV 2012',
    'Jeep Grand Cherokee SUV 2012',
    'Jeep Compass SUV 2012',
    'Lamborghini Reventon Coupe 2008',
    'Lamborghini Aventador Coupe 2012',
    'Lamborghini Gallardo LP 570-4 Superleggera 2012',
    'Lamborghini Diablo Coupe 2001',
    'Land Rover Range Rover SUV 2012',
    'Land Rover LR2 SUV 2012',
    'Lincoln Town Car Sedan 2011',
    'MINI Cooper Roadster Convertible 2012',
    'Maybach Landaulet Convertible 2012',
    'Mazda Tribute SUV 2011',
    'McLaren MP4-12C Coupe 2012',
    'Mercedes-Benz 300-Class Convertible 1993',
    'Mercedes-Benz C-Class Sedan 2012',
    'Mercedes-Benz SL-Class Coupe 2009',
    'Mercedes-Benz E-Class Sedan 2012',
    'Mercedes-Benz S-Class Sedan 2012',
    'Mercedes-Benz Sprinter Van 2012',
    'Mitsubishi Lancer Sedan 2012',
    'Nissan Leaf Hatchback 2012',
    'Nissan NV Passenger Van 2012',
    'Nissan Juke Hatchback 2012',
    'Nissan 240SX Coupe 1998',
    'Plymouth Neon Coupe 1999',
    'Porsche Panamera Sedan 2012',
    'Ram C/V Cargo Van Minivan 2012',
    'Rolls-Royce Phantom Drophead Coupe Convertible 2012',
    'Rolls-Royce Ghost Sedan 2012',
    'Rolls-Royce Phantom Sedan 2012',
    'Scion xD Hatchback 2012',
    'Spyker C8 Convertible 2009',
    'Spyker C8 Coupe 2009',
    'Suzuki Aerio Sedan 2007',
    'Suzuki Kizashi Sedan 2012',
    'Suzuki SX4 Hatchback 2012',
    'Suzuki SX4 Sedan 2012',
    'Tesla Model S Sedan 2012',
    'Toyota Sequoia SUV 2012',
    'Toyota Camry Sedan 2012',
    'Toyota Corolla Sedan 2012',
    'Toyota 4Runner SUV 2012',
    'Volkswagen Golf Hatchback 2012',
    'Volkswagen Golf Hatchback 1991',
    'Volkswagen Beetle Hatchback 2012',
    'Volvo C30 Hatchback 2012',
    'Volvo 240 Sedan 1993',
    'Volvo XC90 SUV 2007',
    'smart fortwo Convertible 2012',
]

convertDict = dict()
for i in range(len(class_names)):
    convertDict[i] = class_names[i]

def classid2name(classid):
    return convertDict[classid]
    

train_dataset = datasets.load_dataset(DATA_REPO_ID, split="train")

test_dataset  = datasets.load_dataset(DATA_REPO_ID, split="test")
# valid_dataset = load_dataset(DATA_REPO_ID, split="validation")


README.md:   0%|          | 0.00/11.0k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/504M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/485M [00:00<?, ?B/s]

test-00000-of-00002.parquet:   0%|          | 0.00/513M [00:00<?, ?B/s]

test-00001-of-00002.parquet:   0%|          | 0.00/474M [00:00<?, ?B/s]

...

Generating impulse_noise split:   0%|          | 0/8041 [00:00<?, ? examples/s]

Generating jpeg_compression split:   0%|          | 0/8041 [00:00<?, ? examples/s]

Generating motion_blur split:   0%|          | 0/8041 [00:00<?, ? examples/s]

Generating pixelate split:   0%|          | 0/8041 [00:00<?, ? examples/s]

Generating spatter split:   0%|          | 0/8041 [00:00<?, ? examples/s]

In [5]:
# prepare collate function

from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("google/paligemma-3b-pt-224")

def train_collate_fn(samples):

    images = []

    for image in [sample["image"] for sample in samples]:
        if image.mode != "RGB":  # Check if the image is not already RGB
            images.append(image.convert("RGB"))
        else:
            images.append(image)

    texts = ["<image><bos>" + "What is the model of a car in the image?" for sample in samples]

    labels = [classid2name(sample["label"])+"<eos>" for sample in samples]



    # define a processor that handle max_length 128(not including the number of image tokens)

    inputs = processor(text=texts, images=images, suffix=labels, return_tensors="pt", 

                      padding=True, truncation=True, max_length=128)



    input_ids = inputs["input_ids"]

    token_type_ids = inputs["token_type_ids"]

    attention_mask = inputs["attention_mask"]

    pixel_values = inputs["pixel_values"]

    labels = inputs["labels"]



    return input_ids, token_type_ids, attention_mask, pixel_values, labels



def test_collate_fn(samples):

    images = []

    for image in [sample["image"] for sample in samples]:
        if image.mode != "RGB":  # Check if the image is not already RGB
            images.append(image.convert("RGB"))
        else:
            images.append(image)

    texts = ["<image><bos>" + "What is the model of a car in the image?" for sample in samples]

    labels = [classid2name(sample["label"]) for sample in samples]



    # define a processor that handle max_length 128(not including the number of image tokens)

    inputs = processor(text=texts, images=images, return_tensors="pt", 

                      padding=True, truncation=True, max_length=128)



    input_ids = inputs["input_ids"]

    attention_mask = inputs["attention_mask"]

    pixel_values = inputs["pixel_values"]



    return input_ids, attention_mask, pixel_values, labels

    

preprocessor_config.json:   0%|          | 0.00/699 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

In [6]:
# TMP TEST PURPOSE
# tmp_train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=train_collate_fn, num_workers=3)

# for i in range(len(train_dataset)):
#     input_ids, token_type_ids, attention_mask, pixel_values, labels = next(iter(tmp_train_dataloader))


## Prepare Quantization Settings

In [7]:
# Using bitsandbytes library, setting the quantization

from transformers import BitsAndBytesConfig

import torch

bnb_config = BitsAndBytesConfig(

    load_in_4bit=True,

    bnb_4bit_compute_dtype=torch.bfloat16, # convert float32 to bf16 to speed up computation

    bnb_4bit_quant_type="nf4", # NF4 is 4-bit data type from QLoRA paper

)

## Prepare LoRA Settings

In [8]:
from peft import LoraConfig


lora_config = LoraConfig(

    r=16, # set the low-rank

    target_modules=["q_proj", "k_proj", "v_proj"], #modules that we are going to apply LoRA adapter

    task_type="CAUSAL_LM", # type of model. PaliGemma is causal language model

    lora_dropout=0.05,
    
    lora_alpha = 32,

)

## Prepare pytorch-lightning Trainer

In [9]:
import lightning as L

from transformers import AutoProcessor

import torch 

from nltk import edit_distance

from torch.utils.data import DataLoader

import numpy as np





class PaliGemma_Finetuned_Model(L.LightningModule):

    def __init__(self, config, model, processor):
        super().__init__()

        self.model = model 

        self.processor = processor

        self.config = config



        self.batch_size = config.get("batch_size")



        self.train_losses = []

        self.val_losses = []

        self.val_scores = []



    def training_step(self, batch, batch_idx):
        input_ids, token_type_ids, attention_mask, pixel_values, labels = batch



        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids,

                            pixel_values=pixel_values, labels=labels)



        loss = outputs.loss

        self.train_losses.append(loss.item())



        self.log("train_loss", loss)

        return loss



    def validation_step(self, batch, batch_idx):

        input_ids, attention_mask, pixel_values, labels = batch



        generated_ids = self.model.generate(input_ids=input_ids, attention_mask=attention_mask, pixel_values=pixel_values, max_new_tokens=80)
        predictions = self.processor.batch_decode(generated_ids[:, input_ids.size(1)+1:], skip_special_tokens=True)
    

        scores = []

        for pred, label in zip(predictions, labels):

            score = edit_distance(pred, label) / max(len(pred), len(label))

            self.val_scores.append(score)

            scores.append(score)

        self.log("val_edit_distance", np.mean(scores), on_epoch=True)

        if self.trainer.sanity_checking:
            print(f"SANITY CHECKING STEP")
            print(f"batch_decoded input_ids: {self.processor.batch_decode(input_ids, skip_special_tokens=True)}")
            print(f"labels: {labels}")
            print(f"prediction: {predictions}")
            print(f"edit_distance_score: {np.mean(scores)}")

        elif batch_idx % 2000 == 0:
            print(f"VALIDATION STEP")
            print(f"batch_decoded input_ids: {self.processor.batch_decode(input_ids, skip_special_tokens=True)}")
            print(f"labels: {labels}")
            print(f"prediction: {predictions}")
            print(f"edit_distance_score: {np.mean(scores)}")
            
            



        return scores



    def configure_optimizers(self):

        optimizer = torch.optim.AdamW(self.parameters(), lr=config.get("lr", 3e-4))

        return optimizer



    def train_dataloader(self):

        return DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True, collate_fn=train_collate_fn, num_workers=3)

    def val_dataloader(self):

        return DataLoader(test_dataset, batch_size=self.batch_size, collate_fn=test_collate_fn, num_workers=3)

        

## Define the model 

In [10]:
from transformers import PaliGemmaForConditionalGeneration

from peft import get_peft_model



model = PaliGemmaForConditionalGeneration.from_pretrained("google/paligemma-3b-pt-224", quantization_config=bnb_config)

model = get_peft_model(model, lora_config)

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/62.6k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [11]:
model.print_trainable_parameters()

trainable params: 5,492,736 || all params: 2,928,959,216 || trainable%: 0.1875


## Setup Training Configurations

In [12]:
config = config = {
            "max_epochs": 100,
            
            "val_check_interval": 0.25, 
            
            "gradient_clip_val": 1.0,
            
            "accumulate_grad_batches": 8,
            
            "lr": 3e-4,
            
            "batch_size": 3,
            
            "seed":1234,
            
            "num_nodes": 1,
            
            "warmup_steps": 50,
            
            "result_path": "./result",
            
            "verbose": True,
            
            "deterministic": True

}



model_module = PaliGemma_Finetuned_Model(config, model, processor)

## Define Callbacks

In [13]:
from lightning.pytorch.callbacks import Callback

from lightning.pytorch.callbacks.early_stopping import EarlyStopping

from huggingface_hub import HfApi



api = HfApi()



FINETUNED_MODEL_ID="ball1433/PaliGemma-StanfordCars-finetuned"



class Print_TrainValidation_ResultCallback(Callback):

    def on_train_epoch_end(self, trainer, pl_module):

        # print the average of training loss 

        print(f'Average Training Loss: {np.mean(pl_module.train_losses)}')



        # print the average of edit distance score

        print(f'Average Validation Score: {np.mean(pl_module.val_scores)}')



        # reset the list

        pl_module.train_losses = []

        pl_module.val_scores = []





class PushToHubCallback(Callback):

    def on_train_epoch_end(self, trainer, pl_module):

        print(f"Pushing model to the hub, epoch {trainer.current_epoch}")

        pl_module.model.push_to_hub(FINETUNED_MODEL_ID,

                                    commit_message=f"Training in progress, epoch {trainer.current_epoch}")



    def on_train_end(self, trainer, pl_module):

        print(f"Pushing model to the hub after training")

        pl_module.processor.push_to_hub(FINETUNED_MODEL_ID,

                                    commit_message=f"Training done")

        pl_module.model.push_to_hub(FINETUNED_MODEL_ID,

                                    commit_message=f"Training done")



early_stop_callback = EarlyStopping(monitor="val_edit_distance", patience=20, verbose=False, mode="min")

## Training

In [ ]:
# define trainer 

trainer = L.Trainer(

        devices=-1, 
        
        accelerator="auto",
        
        max_epochs=config.get("max_epochs"),
        
        accumulate_grad_batches=config.get("accumulate_grad_batches"),
        
        val_check_interval=config.get("val_check_interval"),
        
        gradient_clip_val=config.get("gradient_clip_val"),
        
        precision="16-mixed",
        
        limit_val_batches=1.0,
        
        num_sanity_val_steps=5,
        
        callbacks=[PushToHubCallback(), Print_TrainValidation_ResultCallback(), early_stop_callback],
    
        deterministic=config.get("deterministic"),

)



trainer.fit(model_module)

INFO: Using 16bit Automatic Mixed Precision (AMP)
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO: `Trainer(limit_val_batches=1.0)` was configured so 100% of the batches will be used..
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name  | Type                 | Params | Mode 
-------------------------------------------------------
0 | model | PeftModelForCausalLM | 1.7 B  | train
-------------------------------------------------------
5.5 M     Trainable params
1.7 B     Non-trainable params
1.7 B     Total params
6,925.360 Total estimated model params size (MB)
1352      Modules in train mode
593       Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/site-packages/lightning/pytorch/utilities/data.py:78: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 3. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


SANITY CHECKING STEP
batch_decoded input_ids: ['What is the model of a car in the image?\n', 'What is the model of a car in the image?\n', 'What is the model of a car in the image?\n']
labels: ['AM General Hummer SUV 2000<eos>', 'AM General Hummer SUV 2000<eos>', 'AM General Hummer SUV 2000<eos>']
prediction: ['mer h1', 'mer h1', 'mer h1']
edit_distance_score: 0.8709677419354839
SANITY CHECKING STEP
batch_decoded input_ids: ['What is the model of a car in the image?\n', 'What is the model of a car in the image?\n', 'What is the model of a car in the image?\n']
labels: ['AM General Hummer SUV 2000<eos>', 'AM General Hummer SUV 2000<eos>', 'AM General Hummer SUV 2000<eos>']
prediction: ['mer h2', 'mer h1', 'mer h1']
edit_distance_score: 0.8602150537634409
SANITY CHECKING STEP
batch_decoded input_ids: ['What is the model of a car in the image?\n', 'What is the model of a car in the image?\n', 'What is the model of a car in the image?\n']
labels: ['AM General Hummer SUV 2000<eos>', 'AM Gen

Training: |          | 0/? [00:00<?, ?it/s]

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Validation: |          | 0/? [00:00<?, ?it/s]

VALIDATION STEP
batch_decoded input_ids: ['What is the model of a car in the image?\n', 'What is the model of a car in the image?\n', 'What is the model of a car in the image?\n']
labels: ['AM General Hummer SUV 2000<eos>', 'AM General Hummer SUV 2000<eos>', 'AM General Hummer SUV 2000<eos>']
prediction: ['MER H2 SUT Crew Cab 2007', 'MER H1 SUV 2002', 'MER H1 SUV 2002']
edit_distance_score: 0.6881720430107526


## Inference